# Data preprocessing

In [18]:
import pandas
import numpy as np
import sklearn.linear_model as lm
from sklearn.model_selection import KFold
from sklearn import preprocessing as pre
import random
import matplotlib.pyplot as plt

## Reading data

In [19]:
housing = pandas.read_csv('./housing.csv')

## Transform ocean_proximity column to 5 binary-valued columns

In [20]:
housing['1h_ocean'] = [1 if i=='<1H OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['island'] = [1 if i=='ISLAND' else 0 for i in housing.ocean_proximity.values]
housing['inland'] = [1 if i=='INLAND' else 0 for i in housing.ocean_proximity.values]
housing['near_ocean'] = [1 if i=='NEAR OCEAN' else 0 for i in housing.ocean_proximity.values]
housing['near_bay'] = [1 if i=='NEAR BAY' else 0 for i in housing.ocean_proximity.values]
housing.drop(columns=['ocean_proximity'], inplace=True)

## Filling missing data

Missing total_bedroom values are predicted through linear regression with total_rooms values

In [21]:
notna = housing.total_bedrooms.notna()
model = lm.LinearRegression()
model.fit(housing.total_rooms.values[notna].reshape(-1,1), housing.total_bedrooms.values[notna].reshape(-1,1))
isna = housing.total_bedrooms.isna()
missing_bedrooms = model.predict(housing.total_rooms.values[isna].reshape(-1,1))
housing.total_bedrooms.loc[isna] = np.squeeze(missing_bedrooms)

C:\Users\Percival\AppData\Roaming\Python\Python36\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## The code below is for feature scaling (normalisation):

In [15]:
import pandas as pd
from sklearn import preprocessing
import xlsxwriter

# need to make sure there are only numbers and that there are no Nan/blank cells in the data 
# before running this code
df = pd.read_excel('housing.xls')

standard = preprocessing.StandardScaler().fit(df)
df = standard.transform(df)

workbook = xlsxwriter.Workbook('housing_normalised.xlsx')
worksheet = workbook.add_worksheet()

row = 0
col = 0

for a,b,c,d,e,f,g,h,i,j in (df):
    worksheet.write(row, col,     a)
    worksheet.write(row, col + 1, b)
    worksheet.write(row, col + 2, c)
    worksheet.write(row, col + 3, d)
    worksheet.write(row, col + 4, e)
    worksheet.write(row, col + 5, f)
    worksheet.write(row, col + 6, g)
    worksheet.write(row, col + 7, h)
    worksheet.write(row, col + 8, i)
    worksheet.write(row, col + 8, j)
    row += 1
    
workbook.close()

ModuleNotFoundError: No module named 'xlsxwriter'

## Data Transform using Box Cox Transform

In [29]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
             'total_bedrooms', 'population', 'households', 'median_income', 
             'median_house_value' ]

housing_maxmin = pd.DataFrame(data = housing)
housing_maxmin[numerical] = scaler.fit_transform(housing[numerical])+0.1

# Show an example of a record with scaling applied
display(housing_maxmin.head(n = 5))

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,1h_ocean,island,inland,near_ocean,near_bay
0,0.311155,0.667481,0.884314,0.122331,0.119863,0.108941,0.120556,0.639668,1.002266,0,0,0,0,1
1,0.312151,0.665356,0.492157,0.280503,0.271477,0.167210,0.286976,0.638027,0.808247,0,0,0,0,1
2,0.310159,0.664293,1.100000,0.137260,0.129330,0.113818,0.128943,0.566028,0.795051,0,0,0,0,1
3,0.309163,0.664293,1.100000,0.132352,0.136313,0.115555,0.135849,0.454699,0.772783,0,0,0,0,1
4,0.309163,0.664293,1.100000,0.141330,0.143296,0.115752,0.142427,0.330776,0.774638,0,0,0,0,1


In [30]:
from scipy import stats

median_house_value_bc, maxlog, interval = stats.boxcox(housing.median_house_value, alpha=0.05)
population_bc, maxlog, interval = stats.boxcox(housing.population, alpha=0.05)
housing_median_age_bc, maxlog, interval = stats.boxcox(housing.housing_median_age, alpha=0.05)
total_rooms_bc, maxlog, interval = stats.boxcox(housing.total_rooms, alpha=0.05)
total_bedrooms_bc, maxlog, interval = stats.boxcox(housing.total_bedrooms, alpha=0.05)
households_bc, maxlog, interval = stats.boxcox(housing.households, alpha=0.05)
median_income_bc, maxlog, interval = stats.boxcox(housing.median_income, alpha=0.05)



In [31]:
housing_boxcox = housing_maxmin.copy()

housing_boxcox.drop(columns=['housing_median_age'], inplace=True)
housing_boxcox.drop(columns=['total_rooms'], inplace=True)
housing_boxcox.drop(columns=['total_bedrooms'], inplace=True)
housing_boxcox.drop(columns=['population'], inplace=True)
housing_boxcox.drop(columns=['households'], inplace=True)
housing_boxcox.drop(columns=['median_income'], inplace=True)
housing_boxcox.drop(columns=['median_house_value'], inplace=True)

housing_boxcox['housing_median_age'] = housing_median_age_bc
housing_boxcox['total_rooms'] = total_rooms_bc
housing_boxcox['total_bedrooms'] = total_bedrooms_bc
housing_boxcox['population'] = population_bc
housing_boxcox['households'] = households_bc
housing_boxcox['median_income'] = median_income_bc
housing_boxcox['median_house_value'] = median_house_value_bc


print(housing_boxcox)

       longitude  latitude  1h_ocean  island  inland  near_ocean  near_bay  \
0       0.311155  0.667481         0       0       0           0         1   
1       0.312151  0.665356         0       0       0           0         1   
2       0.310159  0.664293         0       0       0           0         1   
3       0.309163  0.664293         0       0       0           0         1   
4       0.309163  0.664293         0       0       0           0         1   
5       0.309163  0.664293         0       0       0           0         1   
6       0.309163  0.663231         0       0       0           0         1   
7       0.309163  0.663231         0       0       0           0         1   
8       0.308167  0.663231         0       0       0           0         1   
9       0.309163  0.663231         0       0       0           0         1   
10      0.308167  0.664293         0       0       0           0         1   
11      0.308167  0.664293         0       0       0           0

In [32]:
# First, extract the data into arrays
y = housing_boxcox.median_house_value.values.reshape(-1,1)
X = housing_boxcox.drop(columns=['median_house_value'], inplace=False).values
print(X.shape)
print(y.shape)
# Pull out 1000 values into a holdout set
holdout = random.sample(range(0,10640),1000)
X_holdout = X[holdout]
y_holdout = y[holdout]
Xt = np.delete(X, holdout, 0)
yt = np.delete(y, holdout, 0)
print(Xt.shape)
print(yt.shape)
a=0.0
b=0.0
Model = lm.LinearRegression()
# Have to shuffle the data because it is grouped.
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(Xt):
    X_train, X_test = Xt[train_index], Xt[test_index]
    y_train, y_test = yt[train_index], yt[test_index]
    Model.fit(X_train, y_train)
    print('Training error: ' + str(Model.score(X_train, y_train)))
    print('Testing error: ' + str(Model.score(X_test, y_test)))
    a += Model.score(X_train, y_train)/5
    b += Model.score(X_test, y_test)/5
print('Average train error: ' + str(a))
print('Average test error: ' + str(b))  

(20640, 13)
(20640, 1)
(19640, 13)
(19640, 1)
Training error: 0.6950192854831417
Testing error: 0.698951565510875
Training error: 0.6913760490480451
Testing error: 0.7144962378570472
Training error: 0.7020454504260673
Testing error: 0.6711584007009412
Training error: 0.6993723277911571
Testing error: 0.6821187843473098
Training error: 0.692492366302129
Testing error: 0.7094456660247351
Average train error: 0.6960610958101081
Average test error: 0.6952341308881816


## After data transform, please use housing_boxcox as input

In [33]:
# First, extract the data into arrays
y = housing_boxcox.median_house_value.values.reshape(-1,1)
X = housing_boxcox.drop(columns=['median_house_value'], inplace=False).values
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree =2) 
X_poly = poly.fit_transform(X) 
print(X_poly.shape)
Model=lm.LinearRegression()

X = X_poly.copy()

print(X.shape)
print(y.shape)
# Pull out 1000 values into a holdout set
holdout = random.sample(range(0,10640),1000)
X_holdout = X[holdout]
y_holdout = y[holdout]
Xt = np.delete(X, holdout, 0)
yt = np.delete(y, holdout, 0)
print(Xt.shape)
print(yt.shape)
a=0.0
b=0.0
# Have to shuffle the data because it is grouped.
kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(Xt):
    X_train, X_test = Xt[train_index], Xt[test_index]
    y_train, y_test = yt[train_index], yt[test_index]
    Model.fit(X_train, y_train)
    print('Training error: ' + str(Model.score(X_train, y_train)))
    print('Testing error: ' + str(Model.score(X_test, y_test)))
    a += Model.score(X_train, y_train)/5
    b += Model.score(X_test, y_test)/5
print('Average train error: ' + str(a))
print('Average test error: ' + str(b))  

(20640, 105)
(20640, 105)
(20640, 1)
(19640, 105)
(19640, 1)
Training error: 0.756222490081572
Testing error: 0.7735333476228671
Training error: 0.7640987248099919
Testing error: 0.7424375947392476
Training error: 0.7648502982643728
Testing error: 0.7377912022892457
Training error: 0.7580529205493486
Testing error: 0.7659035417821796
Training error: 0.7593595712099659
Testing error: 0.7599273440398667
Average train error: 0.7605168009830503
Average test error: 0.7559186060946814
